In [1]:
import sys
sys.path.append('/home/anokhin/distribution_connector')

In [2]:
from sklearn import datasets
import numpy as np
import torch
from tqdm import tqdm

In [3]:
sys.path

['/home/anokhin/dnn-mode-connectivity',
 '/home/anokhin/anaconda3/lib/python37.zip',
 '/home/anokhin/anaconda3/lib/python3.7',
 '/home/anokhin/anaconda3/lib/python3.7/lib-dynload',
 '',
 '/home/anokhin/anaconda3/lib/python3.7/site-packages',
 '/home/anokhin/anaconda3/lib/python3.7/site-packages/IPython/extensions',
 '/home/anokhin/.ipython',
 '/home/anokhin/distribution_connector']

In [4]:
from connector import Connector
from one_layer_utils import samples, make_dataset, get_model, get_b
from utils import test_model

In [5]:
# # making CIFAR dataset
# import models
# architecture = getattr(models, "LinearOneLayerCF") #LinearOneLayer LogRegression

# dataset, B = make_dataset(model_dir='curves/LinearOneLayer/curve', 
#              dataset='CIFAR', 
#              architecture=architecture, 
#              N_models=21, check=400)

In [6]:
# # making MNIST dataset
# import models
# architecture = getattr(models, "LinearOneLayer") #LinearOneLayer LogRegression

# dataset, B = make_dataset(model_dir='curves_mnist/LinearOneLayer/LongTraining/curve', 
#              dataset='MNIST', 
#              architecture=architecture, 
#              N_models=61, check=30)

In [7]:
# dataset.shape, B.shape

In [50]:
from flows.models.RNVP import RealNVP, RealNVP_OneLayer
from flows.models.IAF import Flow_IAF
flow_RNVP = RealNVP(in_dim=795, dim_middle=1000, N_layers=5, batch_norm=True)
flow_IAF = Flow_IAF(in_dim=795, dim_middle=1000, N_layers=5, batch_norm=True)

flow_RNVP.load_state_dict(torch.load('/home/anokhin/distribution_connector/flow_models/RealNVP/MNIST/checkpoint-100.pt')['model_state'])
flow_IAF.load_state_dict(torch.load('/home/anokhin/distribution_connector/flow_models/Flow_IAF/MNIST/checkpoint-100.pt')['model_state'])

flows = [flow_RNVP, flow_IAF]


mask len 10


In [51]:
import models
architecture = getattr(models, "LinearOneLayer") #LinearOneLayer LogRegression

model1 = architecture.base(num_classes=10, **architecture.kwargs)
model2 = architecture.base(num_classes=10, **architecture.kwargs)

model1.load_state_dict(torch.load('curves_mnist/LinearOneLayer/LongTraining/curve30/checkpoint-30.pt')['model_state'])
model2.load_state_dict(torch.load('curves_mnist/LinearOneLayer/LongTraining/curve40/checkpoint-30.pt')['model_state'])

mnist = [samples(model1), samples(model2)]

import data
loaders, num_classes = data.loaders(
    "MNIST",
    "data",
    1024,
    1,
    "VGG",
    False)


Using train (45000) + validation (5000)
len 55000


In [79]:
def test(dataset, K=80, show=False, verbose=False):
    stat = {}
    stat['train'] = {}
    stat['test'] = {}
    
    flow_ind = 0
    t = np.linspace(0,1, 3)[1:-1]
    if verbose:
        print('True models: ')
        test_model(model1, loaders)
        test_model(model2, loaders)
    cntr = Connector(*dataset)
    funcs = [func for func in dir(cntr) if callable(getattr(cntr, func)) and 'connect' in func] #+ ['flow_connect']
    for f in funcs:
        func = getattr(cntr, f)
        if verbose:
            print(func.__name__)
        if func.__name__=='flow_connect':
            continue
            flow = flows[flow_ind] 
            flow_ind+=1
            print(flow.__class__.__name__)
            res = func(flow, t=t)
        elif func.__name__=='third_cumulant_connect':
            continue
            res = func(t=t, K=80, coef=0.5)
        elif func.__name__=='arc_connect':
#             continue
            res = func(t=t)
        elif 'arc' in func.__name__ and 'PCA' in func.__name__:
#             continue
            res = func(t=t, K=K)
        else:
#             continue
            res = func(t=t)

        b = get_b(model1, model2)
        for ind, r in enumerate(res[1:-1]):
            if verbose:
                print(ind)
            model = get_model(r, b, architecture)
            train_res, test_res = test_model(model, loaders, verbose=verbose)
            stat['train'][func.__name__] = train_res['accuracy']
            stat['test'][func.__name__] = test_res['accuracy']
        if show:
            cntr.show_samples(res, show='hex', limits=False)
    return stat


In [ ]:
mnist_stat = []
for i in tqdm(range(1, 5)):
    for j in range(i+1, 5):
        model1.load_state_dict(torch.load('curves_mnist/LinearOneLayer/LongTraining/curve'+str(30+i)+'/checkpoint-30.pt')['model_state'])
        model2.load_state_dict(torch.load('curves_mnist/LinearOneLayer/LongTraining/curve'+str(30+j)+'/checkpoint-30.pt')['model_state'])

        mnist = [samples(model1), samples(model2)]
        stat = test(mnist, K=70)
        mnist_stat.append(stat)
        
mnist_dict = {'train':{}, 'test':{}}
for k1 in mnist_stat[0].keys():
    for k2 in stat[k1].keys():
        mnist_dict[k1][k2] = []
    
for stat in mnist_stat:
    for k1 in stat.keys():
        for k2 in stat[k1].keys():
            mnist_dict[k1][k2].append(stat[k1][k2])


  0%|          | 0/4 [00:00<?, ?it/s]

RealNVP
RealNVP
RealNVP


In [56]:
mnist_dict = {'train':{}, 'test':{}}
for k1 in mnist_stat[0].keys():
    for k2 in stat[k1].keys():
        mnist_dict[k1][k2] = []
    
for stat in mnist_stat:
    for k1 in stat.keys():
        for k2 in stat[k1].keys():
            mnist_dict[k1][k2].append(stat[k1][k2])

In [57]:
mnist_dict

{'train': {'arc_connect': [97.91636363636364,
   97.81636363636363,
   97.58727272727273,
   97.81272727272727,
   97.7909090909091,
   97.86727272727273],
  'arc_connect_PCA': [97.59454545454545,
   97.5490909090909,
   97.49272727272728,
   97.45454545454545,
   97.48727272727272,
   97.66545454545455],
  'arc_connect_PCA_std': [97.60181818181819,
   97.57272727272728,
   97.49818181818182,
   97.44727272727273,
   97.46363636363637,
   97.66181818181818],
  'arc_connect_std': [97.91636363636364,
   97.81636363636363,
   97.58727272727273,
   97.81272727272727,
   97.7909090909091,
   97.86727272727273],
  'flow_connect': [97.00545454545454,
   96.88181818181818,
   96.23818181818181,
   96.98727272727272,
   96.62181818181818,
   96.92727272727272],
  'inverse_connect': [97.87636363636364,
   97.74909090909091,
   97.58,
   97.84727272727272,
   97.79272727272728,
   97.84],
  'inverse_connect_PCA': [97.66,
   97.48181818181818,
   97.4290909090909,
   97.55818181818182,
   97.55636

In [58]:
for k1 in mnist_stat[0].keys():
    print(k1)
    print('='*10)
    for k2 in stat[k1].keys():
        print(k2)
        print('-'*20)
        print('mean', np.array(mnist_dict[k1][k2]).mean())
        print('2 std', 2*np.array(mnist_dict[k1][k2]).std())

train
arc_connect
--------------------
mean 97.79848484848486
2 std 0.20626460490349757
arc_connect_PCA
--------------------
mean 97.54060606060607
2 std 0.14401816026698105
arc_connect_PCA_std
--------------------
mean 97.5409090909091
2 std 0.15471648290548537
arc_connect_std
--------------------
mean 97.79848484848486
2 std 0.20626460490349757
flow_connect
--------------------
mean 96.77696969696969
2 std 0.5442923645845906
inverse_connect
--------------------
mean 97.78090909090908
2 std 0.19752462621514516
inverse_connect_PCA
--------------------
mean 97.5460606060606
2 std 0.14844897525843914
lin_connect
--------------------
mean 96.60757575757576
2 std 0.7043334317663482
lin_connect_PCA
--------------------
mean 27.463636363636365
2 std 0.3412567923823748
simul_diag_connect
--------------------
mean 97.75999999999999
2 std 0.19002102247044605
third_cumulant_connect
--------------------
mean 80.74787878787879
2 std 3.232230705746402
two_simul_diag_connect
--------------------
mea

In [59]:
# flow = RealNVP(in_dim=3083, dim_middle=1000, N_layers=5, batch_norm=True)
# flow.load_state_dict(torch.load('/home/anokhin/distribution_connector/flow_models/RealNVP/CIFAR/checkpoint-100.pt')['model_state'])

In [72]:
from flows.models.RNVP import RealNVP
from flows.models.IAF import Flow_IAF
flow_RNVP = RealNVP(in_dim=3083, dim_middle=1000, N_layers=5, batch_norm=True)
flow_IAF = Flow_IAF(in_dim=3083, dim_middle=1000, N_layers=5, batch_norm=True)

flow_RNVP.load_state_dict(torch.load('/home/anokhin/distribution_connector/flow_models/RealNVP/CIFAR/checkpoint-100.pt')['model_state'])
flow_IAF.load_state_dict(torch.load('/home/anokhin/distribution_connector/flow_models/Flow_IAF/CIFAR/checkpoint-100.pt')['model_state'])

flows = [flow_RNVP, ] #flow_IAF


mask len 10


In [76]:
import models
architecture = getattr(models, "LinearOneLayerCF") #LinearOneLayer LogRegression

model1 = architecture.base(num_classes=10, **architecture.kwargs)
model2 = architecture.base(num_classes=10, **architecture.kwargs)

model1.load_state_dict(torch.load('curves/LinearOneLayer/curve1/checkpoint-400.pt')['model_state'])
model2.load_state_dict(torch.load('curves/LinearOneLayer/curve2/checkpoint-400.pt')['model_state'])

cifar = [samples(model1), samples(model2)]
test_datasets = [cifar]

import data
loaders, num_classes = data.loaders(
    "CIFAR10",
    "data",
    128,
    1,
    "VGG",
    False)

Files already downloaded and verified
Using train (45000) + validation (5000)
len 45000
Files already downloaded and verified


In [80]:
mnist_stat = []
for i in tqdm(range(1, 5)):
    for j in range(i+1, 5):
        model1.load_state_dict(torch.load('curves/LinearOneLayer/curve'+str(1+i)+'/checkpoint-400.pt')['model_state'])
        model2.load_state_dict(torch.load('curves/LinearOneLayer/curve'+str(1+j)+'/checkpoint-400.pt')['model_state'])

        mnist = [samples(model1), samples(model2)]
        stat = test(cifar, K=300)
        mnist_stat.append(stat)
        
mnist_dict = {'train':{}, 'test':{}}
for k1 in mnist_stat[0].keys():
    for k2 in stat[k1].keys():
        mnist_dict[k1][k2] = []
    
for stat in mnist_stat:
    for k1 in stat.keys():
        for k2 in stat[k1].keys():
            mnist_dict[k1][k2].append(stat[k1][k2])





  0%|          | 0/4 [00:00<?, ?it/s]



 25%|██▌       | 1/4 [28:44<1:26:13, 1724.37s/it]



 50%|█████     | 2/4 [48:01<51:48, 1554.12s/it]  



 75%|███████▌  | 3/4 [57:32<20:59, 1259.38s/it]



100%|██████████| 4/4 [57:32<00:00, 863.22s/it] 

In [82]:
for k1 in mnist_stat[0].keys():
    print(k1)
    print('='*10)
    for k2 in stat[k1].keys():
        print(k2)
        print('-'*20)
        print('mean', np.array(mnist_dict[k1][k2]).mean())
        print('2 std', 2*np.array(mnist_dict[k1][k2]).std())

train
arc_connect
--------------------
mean 49.08592592592593
2 std 0.4062367017977545
arc_connect_PCA
--------------------
mean 49.038148148148146
2 std 0.17500460507364218
arc_connect_PCA_std
--------------------
mean 49.17777777777778
2 std 0.5553303247143282
arc_connect_std
--------------------
mean 49.227037037037036
2 std 0.5727439788377742
inverse_connect
--------------------
mean 40.05148148148148
2 std 0.35541123614259573
inverse_connect_PCA
--------------------
mean 36.13296296296296
2 std 0.5068967422170262
lin_connect
--------------------
mean 30.03222222222222
2 std 0.32386300004531887
lin_connect_PCA
--------------------
mean 10.309259259259258
2 std 0.053605273781868704
simul_diag_connect
--------------------
mean 38.26666666666666
2 std 0.3044363340363334
two_simul_diag_connect
--------------------
mean 11.60074074074074
2 std 0.1968921773267175
test
arc_connect
--------------------
mean 39.68
2 std 0.040000000000001514
arc_connect_PCA
--------------------
mean 40.44666

In [ ]:
# test(K=400)